In [ ]:
# Paths - Update locally!
git_path = '/path/to/git/kurteff2024_code/'
data_path = '/path/to/bids/dataset/'

In [ ]:
import mne
import numpy as np
import pandas as pd
import os
import re
import csv
from tqdm.notebook import tqdm
import warnings

from img_pipe import img_pipe

from matplotlib import pyplot as plt
from matplotlib import rcParams as rc
import matplotlib.patheffects as PathEffects
rc['pdf.fonttype'] = 42
plt.style.use('seaborn')
%matplotlib inline

import sys
sys.path.append(os.path.join(git_path,"figures"))
import plotting_utils

In [ ]:
subjs = [s for s in os.listdir(
    os.path.join(git_path,"preprocessing","events","csv")) if "TCH" in s or "S0" in s]
exclude = ["TCH8"]
no_imaging = ["S0010"]
subjs = [s for s in subjs if s not in exclude]

blocks = {
    s: [
        b.split("_")[-1] for b in os.listdir(os.path.join(
            git_path,"analysis","events","csv",s)) if f"{s}_B" in b and os.path.isfile(os.path.join(
            git_path,"analysis","events","csv",s,b,f"{b}_spkr_sn_all.txt"
        ))
    ] for s in subjs
}

hems = {s:[] for s in subjs}
for s in subjs:
    pt = img_pipe.freeCoG(f"{s}_complete",hem='stereo',subj_dir=data_path)
    elecs = pt.get_elecs()['elecmatrix']
    if sum(elecs[:,0] > 0) >= 1:
        hems[s].append('rh')
    if sum(elecs[:,0] < 0) >= 1:
        hems[s].append('lh')

color_palette = pd.read_csv(os.path.join(git_path,"figures","color_palette.csv"))
spkr_color = color_palette.loc[color_palette['color_id']=='perception']['hex'].values[0]
mic_color = color_palette.loc[color_palette['color_id']=='production']['hex'].values[0]

### Load in onset dataframe

In [ ]:
elec_types = {s:{'spkr_only':[],'mic_only':[],'dual_onset':[],'no_onset':[]} for s in subjs}
onset_df = pd.read_csv(os.path.join(git_path,"stats","onset_stats.csv"))
for s in np.unique(onset_df['subj']):
    ch_names = np.unique(onset_df.loc[onset_df['subj']==s]['ch_name'])
    for ch in ch_names:
        clip_df = onset_df.loc[(onset_df['subj']==s)&(onset_df['ch_name']==ch)]
        if np.isnan(clip_df.loc[clip_df['condition']=='spkr']['peak_amplitude'].values[0]):
            if np.isnan(clip_df.loc[clip_df['condition']=='mic']['peak_amplitude'].values[0]):
                elec_types[s]['no_onset'].append(ch)
            else:
                elec_types[s]['mic_only'].append(ch)
        elif np.isnan(clip_df.loc[clip_df['condition']=='mic']['peak_amplitude'].values[0]):
            elec_types[s]['spkr_only'].append(ch)
        else:
            elec_types[s]['dual_onset'].append(ch)

In [ ]:
# Normalize peak amplitude values
# These values between 0-1 will provide the color scale for our cmaps
# Dual onset will use a normalized ∆Z
peak_amps = dict()
for k in ['dual_onset','spkr_only','mic_only']:
    kdf = []
    for s in subjs:
        ch_rows = [onset_df.loc[onset_df['subj']==s].loc[onset_df['ch_name']==ch] for ch in elec_types[s][k]]
        if len(ch_rows) > 0:
            kdf.append(pd.concat(ch_rows))
    kdf = pd.concat(kdf)
    if "_only" in k:
        kdf = kdf.loc[kdf['condition']==k.replace("_only","")]
    peak_amps[k] = pd.DataFrame(columns=['subj','ch_name','hem','condensed_roi_md','value_norm','value_native'])
    for s in np.unique(kdf['subj']):
        sdf = kdf.loc[kdf['subj']==s]; ch_names = np.unique(sdf['ch_name'])
        for ch in ch_names:
            cdf = sdf.loc[sdf['ch_name']==ch]; hem = cdf['hem'].values[0]; roi = cdf['condensed_roi_md'].values[0]
            if len(cdf) > 1:
                if k == 'dual_onset':
                    mic_peak_amplitude = cdf.loc[cdf['condition']=='mic']['peak_amplitude'].values[0]
                    spkr_peak_amplitude = cdf.loc[cdf['condition']=='spkr']['peak_amplitude'].values[0]
                    value_native = spkr_peak_amplitude - mic_peak_amplitude; value_norm = 0.
            else:
                # Either spkr only or mic only
                value_native = cdf['peak_amplitude'].values[0]; value_norm = 0.
            new_row = pd.DataFrame({'subj':[s], 'ch_name':[ch], 'hem':[hem], 'condensed_roi_md':[roi],
                                    'value_norm':[value_norm], 'value_native':[value_native]})
            peak_amps[k] = peak_amps[k].append(new_row,ignore_index=True)
    if k not in  ["dual_onset"]:
        # Have to normalize the values
        kmin = peak_amps[k]['value_native'].min(); kmax = peak_amps[k]['value_native'].max()
        normed_values = ((peak_amps[k]['value_native'].values-kmin)/(kmax-kmin))
        peak_amps[k]['value_norm'] = normed_values
    elif k == "dual_onset":
        # This is a diverging colormap so we have to normalize the positive and negative values
        # separately to keep 0.5 as the midpoint
        kmin_pos = peak_amps[k].loc[peak_amps[k]['value_native']>=0]['value_native'].min()
        kmax_pos = peak_amps[k].loc[peak_amps[k]['value_native']>=0]['value_native'].max()
        kmin_neg = peak_amps[k].loc[peak_amps[k]['value_native']<=0]['value_native'].min()
        kmax_neg = peak_amps[k].loc[peak_amps[k]['value_native']<=0]['value_native'].max()
        for i,row in peak_amps[k].iterrows():
            value_native = row['value_native']
            if value_native >= 0:
                # Normalize between 0.5 and 1.
                peak_amps[k].at[i,'value_norm'] = ((value_native-kmin_pos)/(kmax_pos-kmin_pos)) * 0.5 + 0.5
            else:
                # Normalize between 0 and 0.5
                peak_amps[k].at[i,'value_norm'] = ((value_native-kmin_neg)/(kmax_pos-kmin_neg)) * 0.5

In [ ]:
mic_cmap = LinearSegmentedColormap.from_list('my_gradient', (
    (0.000, (1.00, 1.000, 1.000)),
    (0.700, (0.584, 0.286, 0.592)),
    (1.000, (0.584, 0.286, 0.592))))
spkr_cmap = LinearSegmentedColormap.from_list('my_gradient', (
    (0.000, (1.00, 1.000, 1.000)),
    (0.700, (0.067, 0.463, 0.196)),
    (1.000, (0.067, 0.463, 0.196))))
dual_onset_cmap = LinearSegmentedColormap.from_list('my_gradient', (
    # Edit this gradient at https://eltos.github.io/gradient/#0:954997-15:954997-40:FFFFFF-50:FFFFFF-60:FFFFFF-85:117632-100:117632
    (0.000, (0.584, 0.286, 0.592)),
    (0.150, (0.584, 0.286, 0.592)),
    (0.400, (1.000, 1.000, 1.000)),
    (0.500, (1.000, 1.000, 1.000)),
    (0.600, (1.000, 1.000, 1.000)),
    (0.850, (0.067, 0.463, 0.196)),
    (1.000, (0.067, 0.463, 0.196))))

In [ ]:
# Save to csv
spkr_df = pd.DataFrame(columns=['subj','hem','ch_name','x','y','z','r','g','b','a'])
mic_df = pd.DataFrame(columns=['subj','hem','ch_name','x','y','z','r','g','b','a'])
dual_onset_df = pd.DataFrame(columns=['subj','hem','ch_name','x','y','z','r','g','b','a'])

for hem in ['lh','rh']:
    for s in [ss for s in subjs if hem in hems[ss]]:
        blockid = "_".join([s,blocks[s][0]])
        fif_ch_names = mne.io.read_raw_fif(os.path.join(data_path,f"sub-{s}",blockid,"HilbAA_70to150_8band",
            "ecog_hilbAA70to150.fif"), preload=False, verbose=False).info['ch_names']
        pt = img_pipe.freeCoG(f'{s}_complete',hem=hem, subj_dir=ip)
        e, a = imaging_utils.clip_4mm_elecs(pt,hem=hem,elecfile_prefix="TDT_elecs_all_warped")
        e, a = imaging_utils.clip_outside_brain_elecs(pt,elecmatrix=e,anatomy=a,hem=hem,
                                                      elecfile_prefix="TDT_elecs_all_warped")
        fs_ch_names = [aa[0][0] for aa in a]
        for elecfile_idx, ch in enumerate(fs_ch_names):
            if ch.replace('-','') in [c.replace('-','') for c in fif_ch_names]:
                fif_idx = [c.replace('-','') for c in fif_ch_names].index(ch.replace('-',''))
                x,y,z = e[elecfile_idx,:]
                if ch in elec_types[s]['spkr_only']:
                    value_norm = peak_amps['spkr_only'].loc[(peak_amps['spkr_only']['subj']==s)&(
                        peak_amps['spkr_only']['ch_name']==ch)]['value_norm'].values[0]
                    r,g,b,a = spkr_cmap(value_norm)
                    new_row = pd.DataFrame({'subj':[s],'hem':[hem],'ch_name':[ch],'x':[x],'y':[y],'z':[z],
                                            'r':[r],'g':[g],'b':[b],'a':[a]})
                    spkr_df = spkr_df.append(new_row, ignore_index=True)
                elif ch in elec_types[s]['mic_only']:
                    value_norm = peak_amps['mic_only'].loc[(peak_amps['mic_only']['subj']==s)&(
                        peak_amps['mic_only']['ch_name']==ch)]['value_norm'].values[0]
                    r,g,b,a = mic_cmap(value_norm)
                    new_row = pd.DataFrame({'subj':[s],'hem':[hem],'ch_name':[ch],'x':[x],'y':[y],'z':[z],
                                            'r':[r],'g':[g],'b':[b],'a':[a]})
                    mic_df = mic_df.append(new_row, ignore_index=True)
                elif ch in elec_types[s]['dual_onset']:
                    value_norm = peak_amps['dual_onset'].loc[(peak_amps['dual_onset']['subj']==s)&(
                        peak_amps['dual_onset']['ch_name']==ch)]['value_norm'].values[0]
                    r,g,b,a = dual_onset_cmap(value_norm)
                    new_row = pd.DataFrame({'subj':[s],'hem':[hem],'ch_name':[ch],'x':[x],'y':[y],'z':[z],
                                            'r':[r],'g':[g],'b':[b],'a':[a]})
                    dual_onset_df = dual_onset_df.append(new_row, ignore_index=True)
spkr_df.to_csv(os.path.join(git_path,"figures","figure_2","csv","figure_2_cmap_spkr.csv"),index=False)
mic_df.to_csv(os.path.join(git_path,"figures","figure_2","csv","figure_2_cmap_mic.csv"),index=False)
dual_onset_df.to_csv(os.path.join(git_path,"figures","figure_2","csv","figure_2_cmap_dual_onset.csv"),index=False)